In [1]:
# import libraries
import sqlite3
import pandas as pd
import os
import ast
from datetime import datetime

In [2]:
# setup sqlite connection
con = sqlite3.connect('../airflow/airflow.db')
cur = con.cursor()

In [ ]:
# prepare player files and load into df
file_list = os.listdir('../RawText')
game_files = [x for x in file_list if x.startswith('games')]
li = []
for file in game_files:
    df=pd.read_csv(f'../RawText/{file}')
    li.append(df)
games_raw = pd.concat(li)

In [ ]:
# extract teamid from team dictionary
games_raw['HomeTeamID'] = games_raw.apply(lambda row: ast.literal_eval(row['home_team'])['id'], axis=1)
games_raw['AwayTeamID'] = games_raw.apply(lambda row: ast.literal_eval(row['visitor_team'])['id'], axis=1)
games_raw['date_clean'] = games_raw.apply(lambda row: row['date'].split('T')[0], axis=1)

In [ ]:
games_raw_load = games_raw[['id', 'date_clean', 'HomeTeamID', 'home_team_score', 'AwayTeamID', 'visitor_team_score', 'postseason', 'season']]

In [ ]:
games_raw_load.head(100)

In [4]:
# prepare second game files with attendance and load into df
file_list = os.listdir('../RawText')
game_files2 = [x for x in file_list if x.startswith('sportsref')]
li = []
for file in game_files2:
    df=pd.read_excel(f'../RawText/{file}')
    li.append(df)
games_raw2 = pd.concat(li)

In [5]:
games_raw2['date_clean'] = games_raw2.apply(lambda row: datetime.strftime(datetime.strptime(row['Date'], '%a, %b %d, %Y'), '%Y-%m-%d'), axis=1)

In [6]:
games_raw_load2 = games_raw2[['date_clean', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1', 'Attend.']]

In [7]:
games_raw_load2.head()

,date_clean,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Attend.
0,2020-12-22,Golden State Warriors,99.0,Brooklyn Nets,125.0,0.0
1,2020-12-22,Los Angeles Clippers,116.0,Los Angeles Lakers,109.0,0.0
2,2020-12-23,Charlotte Hornets,114.0,Cleveland Cavaliers,121.0,300.0
3,2020-12-23,New York Knicks,107.0,Indiana Pacers,121.0,0.0
4,2020-12-23,Miami Heat,107.0,Orlando Magic,113.0,3396.0


In [8]:
# write to sql database
games_raw_load2.to_sql(name = 'games2', con=con, index=False, if_exists='append')

In [9]:
# finalize and close connections
con.commit()
cur.close()
con.close()